# 패키지 설치하기

In [ ]:
!pip install ratsnlp

# 환경 설정

모델 hyperparameter와 저장 위치 등 설정 정보를 선언

In [2]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

In [3]:
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/content/drive/Othercomputers/내 컴퓨터/Chapter 4. Document Classification/checkpoint-doccls",
    max_seq_length=128,
)

downstream_model_checkpoint_fpath: /content/drive/Othercomputers/내 컴퓨터/Chapter 4. Document Classification/checkpoint-doccls/epoch=1-val_loss=0.26-v1.ckpt


# 모델 불러오기

파인튜닝을 마친 모델 불러오기

In [4]:
import torch
from transformers import BertConfig, BertForSequenceClassification

In [5]:
# 체크포인트 로드
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

# BERT 설정 로드
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForSequenceClassification(pretrained_model_config)  # 모델 초기화
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})     # 체크포인트 주입
model.eval()    # 평가모드

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

토크나이저 준비

In [6]:
from transformers import BertTokenizer

In [7]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 인퍼런스 함수

In [8]:
def inference_fn(sentence):
    
    # 입력 문장을 토큰화
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )

    # 평가 모드로 Gradient 업데이트 안함
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})  # 토큰화 된 입력 문장을 텐서로 변환 후 모델에 입력
        prob = outputs.logits.softmax(dim=1)
        
        # 긍정, 부정 확률을 소수점 4자리로 반올림
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"

    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

# 웹서비스 개시

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구이다.
실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 입력해야 한다.

In [11]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

 인퍼런스 함수를 Flask를 통해 웹서비스로 만든다.

In [12]:
from ratsnlp.nlpbook.classification import get_web_service_app

In [13]:
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a63e-34-80-252-252.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Apr/2022 12:57:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:57:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:57:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:57:20] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2022 12:57:41] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:57:45] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:57:55] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:58:01] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:58:06] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:58:32] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:58:41] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:59:10] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:59:34] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:59:41] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 12:59:48] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 13:00:02] "POST /a